# Dependencies

- math module

# Traditional Bond

$$
PV = - (C[\frac{1 - \frac{1}{(1+y)^N}}{y}] + \frac{FV}{(1+y)^N})
$$

## Yield to Maturity (YTM)

Using the Newton-Raphson method: 
$$y_{n+1} = y_n - \frac{F(y_n)}{F'(y_n)}$$
where 
$$ F(y) = \frac{C}{1+y} + \frac{C}{(1+y)^2} + ... + \frac{C}{(1+y)^N} + \frac{FV}{(1+y)^N} + PV \\ = C[ \frac{1 - \frac{1}{(1 + y)^N} }{y}] + \frac{FV}{(1+y)^N} + PV $$
and
$$ F'(y) = -\frac{C}{(1+y)^2} - \frac{2C}{(1+y)^3} - ... - \frac{N * C}{(1 + y)^{N+1}} - \frac {N * FV}{(1+y)^{N+1}} \\ = -C \{ \frac{1}{y^2}[1 - \frac{1}{(1 + y)^N}] - \frac{N}{y (1 + y)^{N+1}} \}$$

In [1]:
def F(y, c, fv, n, pv):
    return c * ( (1 - 1 / ( (1 + y) ** n ) ) / y ) + fv / ( (1 + y) ** n ) + pv

In [2]:
def dF(y, c, fv, n, pv):
    return -c * ( (1 / y ** 2) * (1 -  1 / (1 + y) ** n ) ) - n / (y * (1 + y) ** (n + 1))

In [3]:
def YTM(c, fv, n, pv, itr=1000):
    """
        c: periodic coupon rate, or payment
        fv: future value of the series of cash flows
        n: number of periods
        pv: present value of the cash flow
        
        The resulting YTM is a periodic rate.
    """
    y = c / fv
    for i in range(itr):
        y = y - F(y, c, fv, n, pv) / dF(y, c, fv, n, pv)
    return y * 100

In [4]:
YTM(c=10, fv=1000, n=10, pv=-900)

2.1202829427100416

## Present Value (PV) and Future Value (FV)

In [5]:
def PV(c, fv, n, y):
    y /= 100
    return - (c * (1 - 1 / (1+y) ** n) / y + fv / (1+y) ** n)

In [6]:
def FV(c, pv, n, y):
    y /= 100
    return (1 + y) ** n * (-pv - c / y) + c / y

In [7]:
PV(10, 1000, 10, 2.1202829427100416)

-899.9999999999957

In [8]:
FV(10, -900, 10, 2.1202829427100416)

1000.0000000000053

## Number of Periods (N)

$$
N = \frac{ln(\frac{FV - C/y}{-PV - C/y})}{ln(1+y)}
$$

In [9]:
import math

In [10]:
def N(c, pv, fv, y):
    y /= 100
    return math.log((fv - c/y) / (-pv - c/y)) / math.log(1+y)

In [11]:
N(10, -900, 1000, 2.1202829427100416)

9.999999999999515

# Bootstrap Pricing

Suppose we are given a list specifying YTMs of bonds with different maturities, and the same face value.  Based on the info, we want to construct a list of spot rates, as well as a list of one-period forward rates.

Some constraints on the given list:
- We assume semi-annual periods;
- The list should have values at the start and the end, otherwise we cannot apply linear interpolation;
- 'None' implies that interpolation should be applied to the entry where the object is located.

Other notes:
- We can, with no doubt, make use of numpy in linear interpolation, but that would be unnecessary for lightweight usage.
- Assume without loss of generality that Par = $1,000.  In fact, whatever par we use, the result will be the same.

In [14]:
def linear_interpolation(ys):
    if len(ys) == 0:
        raise Exception('An empty list is provided.')
    if ys[0] is None or ys[-1] is None:
        raise Exception('')
    start = -1
    while start < len(ys):
        while start < len(ys)-1 and ys[start+1] is not None:
            start += 1
        if start == len(ys)-1:
            break
        
        end = start + 1
        while end < len(ys) and ys[end] is None:
            end += 1
        if end == len(ys):
            break
        # fill in the points between start and end
        delta = (ys[end] - ys[start]) / (end - start)
        for i in range(end - start - 1):
            ys[start+i+1] = ys[start+i] + delta
        # continue
        start = end
    return ys

In [11]:
ys_test = [1, 2, 3, None, 4, 8, None, None, 10]

In [15]:
linear_interpolation([])

Exception: An empty list is provided.